In [8]:
from darts.tools.keyword_file_tools import load_single_keyword
import numpy as np

In [9]:
nx, ny, nz = 62, 62, 18 + 25 + 18
nb = nx * ny * nz
mz = 25         # middle of the reservoir
n_conf = (nz - mz) // 2   
upper_idx = range(0, n_conf)  
middle_idx = range(n_conf, n_conf + mz) 
lower_idx = range(n_conf + mz, nz) 

dx = np.hstack([[200]*5, [100]*5, [50]*3, [50]*36, [50]*3, [100]*5, [200]*5]).astype(float)
dy = dx.copy()
dz = np.hstack([
    [30,30,30,30,30,30,30,30,30,20,20,10,10,10,6,6,4,4],
    [4]*25,
    [4,4,6,6,10,10,10,20,20,30,30,30,30,30,30,30,30,30]
]).astype(float)

dx = np.broadcast_to(dx[:,None,None], (nx,ny,nz))
dy = np.broadcast_to(dy[None,:,None], (nx,ny,nz))
dz = np.broadcast_to(dz[None,None,:], (nx,ny,nz))

perm0 = load_single_keyword('HeterogeneousModel/largeModel/area1800_varRcond_0_perm.txt', 'PERM')
perm1 = load_single_keyword('HeterogeneousModel/largeModel/area1800_varRcond_1_perm.txt', 'PERM')
perm2 = load_single_keyword('HeterogeneousModel/largeModel/area1800_varRcond_2_perm.txt', 'PERM')
perm3 = load_single_keyword('HeterogeneousModel/largeModel/area1800_varRcond_3_perm.txt', 'PERM')
perm4 = load_single_keyword('HeterogeneousModel/largeModel/area1800_varRcond_4_perm.txt', 'PERM')

poro0 = load_single_keyword('HeterogeneousModel/largeModel/area1800_varRcond_0_poro.txt', 'PORO')
poro1 = load_single_keyword('HeterogeneousModel/largeModel/area1800_varRcond_1_poro.txt', 'PORO')
poro2 = load_single_keyword('HeterogeneousModel/largeModel/area1800_varRcond_2_poro.txt', 'PORO')
poro3 = load_single_keyword('HeterogeneousModel/largeModel/area1800_varRcond_3_poro.txt', 'PORO')
poro4 = load_single_keyword('HeterogeneousModel/largeModel/area1800_varRcond_4_poro.txt', 'PORO')

Reading PERM from c:\Users\kajab\Documents\School\Master_year_2\Thesis\MSc-Thesis-Kaj\HeterogeneousModel\largeModel\area1800_varRcond_0_perm.txt... 234484 values have been read.
Reading PERM from c:\Users\kajab\Documents\School\Master_year_2\Thesis\MSc-Thesis-Kaj\HeterogeneousModel\largeModel\area1800_varRcond_1_perm.txt... 234484 values have been read.
Reading PERM from c:\Users\kajab\Documents\School\Master_year_2\Thesis\MSc-Thesis-Kaj\HeterogeneousModel\largeModel\area1800_varRcond_2_perm.txt... 234484 values have been read.
Reading PERM from c:\Users\kajab\Documents\School\Master_year_2\Thesis\MSc-Thesis-Kaj\HeterogeneousModel\largeModel\area1800_varRcond_3_perm.txt... 234484 values have been read.
Reading PERM from c:\Users\kajab\Documents\School\Master_year_2\Thesis\MSc-Thesis-Kaj\HeterogeneousModel\largeModel\area1800_varRcond_4_perm.txt... 234484 values have been read.
Reading PORO from c:\Users\kajab\Documents\School\Master_year_2\Thesis\MSc-Thesis-Kaj\HeterogeneousModel\large

In [11]:
def effective_k(perm, layer_idx, layer):
    perm = perm[:nb]
    perm = perm.reshape((nx, ny, nz), order='F') 
    sub = perm[:, :, layer_idx]
    check = np.unique(sub)
    if check.size == 1:
        print(f"[{layer}] uniform k = {check[0]} mD")
        return check[0]
    
    print(f"[{layer}] stats → "
      f"min={sub.min():.2f}, max={sub.max():.2f}, "
      f"mean={sub.mean():.2f}, median={np.median(sub):.2f}, std={sub.std():.2f}")
    harmonic_layer = []
    vol = []
    for k in layer_idx:
        A = dx[:, :, k] * dy[:, :, k]
        perm_ij = perm[:, :, k]
        harmonic_layer.append(A.sum() / (A / perm_ij).sum())
        vol.append(A.sum() * dz[0, 0, k]) 
    harmonic_layer = np.array(harmonic_layer)
    vol = np.array(vol)
    k_eff    = (harmonic_layer * vol).sum() / vol.sum()

    print(f"[{layer}] upscaled k = {k_eff} mD")
    return k_eff

def effective_phi(poro, layer_idx, layer):
    poro = poro[:nb]
    poro = poro.reshape((nx, ny, nz), order='F') 
    sub = poro[:, :, layer_idx]
    check = np.unique(sub)
    if check.size == 1:
        print(f"[{layer}] uniform phi = {check[0]}")
        return check[0]
    
    print(f"[{layer}] stats → "
      f"min={sub.min():.2f}, max={sub.max():.2f}, "
      f"mean={sub.mean():.2f}, median={np.median(sub):.2f}, std={sub.std():.2f}")
    vol = dx[:, :, layer_idx] * dy[:, :, layer_idx] * dz[:, :, layer_idx]
    vol = np.array(vol)
    phi = poro[:, :, layer_idx]
    phi = np.array(phi)
    phi_eff = (phi * vol).sum() / vol.sum()
    print(f"[{layer}] upscaled phi = {phi_eff:.2f}")
    return phi_eff

In [12]:
k_eff_upper = effective_k(perm0, upper_idx, "Model 0, Upper confining")
k_eff_middle = effective_k(perm0, middle_idx, "Model 0, Middle layers")
k_eff_lower = effective_k(perm0, lower_idx, "Model 0, Lower confining")
k_eff_upper1 = effective_k(perm1, upper_idx, "Model 1, Upper confining")
k_eff_middle1 = effective_k(perm1, middle_idx, "Model 1, Middle layers")
k_eff_lower1 = effective_k(perm1, lower_idx, "Model 1, Lower confining")
k_eff_upper2 = effective_k(perm2, upper_idx, "Model 2, Upper confining")
k_eff_middle2 = effective_k(perm2, middle_idx, "Model 2, Middle layers")
k_eff_lower2 = effective_k(perm2, lower_idx, "Model 2, Lower confining")
k_eff_upper3 = effective_k(perm3, upper_idx, "Model 3, Upper confining")
k_eff_middle3 = effective_k(perm3, middle_idx, "Model 3, Middle layers")
k_eff_lower3 = effective_k(perm3, lower_idx, "Model 3, Lower confining")
k_eff_upper4 = effective_k(perm4, upper_idx, "Model 4, Upper confining")
k_eff_middle4 = effective_k(perm4, middle_idx, "Model 4, Middle layers")
k_eff_lower4 = effective_k(perm4, lower_idx, "Model 4, Lower confining")

[Model 0, Upper confining] uniform k = 0.001 mD
[Model 0, Middle layers] stats → min=1.00, max=2002.78, mean=220.38, median=2.04, std=450.33
[Model 0, Middle layers] upscaled k = 2.425577331181285 mD
[Model 0, Lower confining] uniform k = 0.001 mD
[Model 1, Upper confining] uniform k = 0.001 mD
[Model 1, Middle layers] stats → min=1.00, max=2002.78, mean=213.63, median=1.82, std=446.92
[Model 1, Middle layers] upscaled k = 2.0549195950204266 mD
[Model 1, Lower confining] uniform k = 0.001 mD
[Model 2, Upper confining] uniform k = 0.001 mD
[Model 2, Middle layers] stats → min=1.00, max=2002.78, mean=228.89, median=2.03, std=452.72
[Model 2, Middle layers] upscaled k = 2.311067600873036 mD
[Model 2, Lower confining] uniform k = 0.001 mD
[Model 3, Upper confining] uniform k = 0.001 mD
[Model 3, Middle layers] stats → min=1.00, max=2002.78, mean=198.50, median=3.13, std=415.85
[Model 3, Middle layers] upscaled k = 2.174924472311165 mD
[Model 3, Lower confining] uniform k = 0.001 mD
[Model 

In [13]:
phi_eff_upper = effective_phi(poro0, upper_idx, "Model 0, Upper confining")
phi_eff_middle = effective_phi(poro0, middle_idx, "Model 0, Middle layers")
phi_eff_lower = effective_phi(poro0, lower_idx, "Model 0, Lower confining")
phi_eff_upper1 = effective_phi(poro1, upper_idx, "Model 1, Upper confining")
phi_eff_middle1 = effective_phi(poro1, middle_idx, "Model 1, Middle layers")
phi_eff_lower1 = effective_phi(poro1, lower_idx, "Model 1, Lower confining")
phi_eff_upper2 = effective_phi(poro2, upper_idx, "Model 2, Upper confining")
phi_eff_middle2 = effective_phi(poro2, middle_idx, "Model 2, Middle layers")
phi_eff_lower2 = effective_phi(poro2, lower_idx, "Model 2, Lower confining")
phi_eff_upper3 = effective_phi(poro3, upper_idx, "Model 3, Upper confining")
phi_eff_middle3 = effective_phi(poro3, middle_idx, "Model 3, Middle layers")
phi_eff_lower3 = effective_phi(poro3, lower_idx, "Model 3, Lower confining")
phi_eff_upper4 = effective_phi(poro4, upper_idx, "Model 4, Upper confining")
phi_eff_middle4 = effective_phi(poro4, middle_idx, "Model 4, Middle layers")
phi_eff_lower4 = effective_phi(poro4, lower_idx, "Model 4, Lower confining")

[Model 0, Upper confining] uniform phi = 0.001
[Model 0, Middle layers] stats → min=0.01, max=0.25, mean=0.10, median=0.09, std=0.06
[Model 0, Middle layers] upscaled phi = 0.11
[Model 0, Lower confining] uniform phi = 0.001
[Model 1, Upper confining] uniform phi = 0.001
[Model 1, Middle layers] stats → min=0.01, max=0.25, mean=0.10, median=0.09, std=0.06
[Model 1, Middle layers] upscaled phi = 0.10
[Model 1, Lower confining] uniform phi = 0.001
[Model 2, Upper confining] uniform phi = 0.001
[Model 2, Middle layers] stats → min=0.01, max=0.25, mean=0.10, median=0.09, std=0.07
[Model 2, Middle layers] upscaled phi = 0.10
[Model 2, Lower confining] uniform phi = 0.001
[Model 3, Upper confining] uniform phi = 0.001
[Model 3, Middle layers] stats → min=0.01, max=0.25, mean=0.10, median=0.09, std=0.06
[Model 3, Middle layers] upscaled phi = 0.10
[Model 3, Lower confining] uniform phi = 0.001
[Model 4, Upper confining] uniform phi = 0.001
[Model 4, Middle layers] stats → min=0.01, max=0.25, 